In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
# import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime

0it [00:00, ?it/s]


In [22]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [ ]:
key = pd.read_csv('./key_1.csv')

In [37]:
sub = pd.read_csv("./sample_submission_1.csv")

In [38]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

NameError: name 'train' is not defined

In [39]:
def get_median(row, last_n_days):
    return pd.rolling_median(pd.Series(row), window = last_n_days)    

In [40]:
def get_mean(row, last_n_days):
    return pd.rolling_mean(pd.Series(row), window = last_n_days)    

In [41]:
def get_long_stats(func_type, last_n_days, df, df_melt):

    if func_type == "median":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_median, last_n_days = last_n_days, axis = 1)
    elif func_type == "mean":
        rolling_stats = df.iloc[:, -60:].progress_apply(get_mean, last_n_days = last_n_days, axis = 1)

    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)    
    rolling_stats_df.columns = ["Page"] + list(df.columns[-60:])
    
    rolling_stats_melt = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))
    
    df_melt = pd.merge(df_melt, rolling_stats_melt, on = ["Page", "date"])     
    return df_melt

In [10]:
train_melt = pd.melt(pd.concat([train.Page, train.iloc[:, -60:]], axis = 1), id_vars=['Page'], var_name="date", value_name="visits")

train_melt = get_long_stats("mean", 7, train, train_melt)
train_melt = get_long_stats("mean", 30, train, train_melt)
train_melt = get_long_stats("mean", 60, train, train_melt)

train_melt = get_long_stats("median", 7, train, train_melt)
train_melt = get_long_stats("median", 30, train, train_melt)
train_melt = get_long_stats("median", 60, train, train_melt)

  0%|          | 0/145063 [00:00<?, ?it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=7,center=False).mean()
  
145064it [01:01, 2365.82it/s]                            
  0%|          | 0/145063 [00:00<?, ?it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=30,center=False).mean()
  
145064it [01:00, 2387.78it/s]                            
  0%|          | 0/145063 [00:00<?, ?it/s]/Library/Python/2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=60,center=False).mean()
  
145064it [00:59, 2435.19it/s]                            
  0%|          | 0/145063 [00:00<?, ?it/s

In [42]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [43]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [44]:
def get_month(row):
    return row.date.split("-")[1]

In [45]:
def get_date(row):
    return row.date.split("-")[2]

In [34]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [9]:
train_melt["month"] = train_melt.progress_apply(get_month, axis = 1)
train_melt["date_num"] = train_melt.progress_apply(get_date, axis = 1)
train_melt["weekday"] = train_melt.progress_apply(get_weekday, axis = 1)
train_melt["language"] = train_melt.progress_apply(get_language, axis = 1)

8703780it [06:46, 21402.26it/s]


In [28]:
# def get_language_mean(row):
lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
print lang_count

language
en                                                                                                     1393140
ja                                                                                                     1220940
de                                                                                                     1074780
fr                                                                                                     1058940
zh                                                                                                     1029120
ru                                                                                                      893220
es                                                                                                      838140
www                                                                                                     413280
commons                                                                                                

In [148]:
def get_test_feats(row, last_index):
    
    subset_row = row.iloc[last_index - 60 : last_index].tolist()
    
    median_7 = np.median(subset_row[-7:])
    median_30 = np.median(subset_row[-30:])
    median_60 = np.median(subset_row[-60:])

    mean_7 = np.mean(subset_row[-7:])
    mean_30 = np.mean(subset_row[-30:])
    mean_60 = np.mean(subset_row[-60:])
    
    return pd.Series([row.Page, median_7, median_30, median_60, mean_7, mean_30, mean_60])

In [149]:
def get_preds(test_feats):
    
    return pd.Series([1 for i in range(0, len(test_feats))])

In [ ]:
# def get_testing_df():
    
# key = pd.read_csv("key_trunc.csv")
# # key["date"] = key.progress_apply(lambda x: x["Page"][-10:], axis = 1)
# key.drop("Page", axis = 1, inplace=True)
# key.to_csv("key_trunc.csv", index=False)

# key_wide = pd.DataFrame()
# key_wide["page"] = np.unique(key.trunc_page).tolist()

# for i in np.unique(key.date).tolist():
#     key_wide[i] = 0

# overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
# overall_df.drop("page", axis = 1, inplace = True)

o_df_h = overall_df.head()

for i in range(overall_df.columns.tolist().index('2017-01-01'), len(overall_df.columns)):
    
    last_date = overall_df.columns.tolist()[i]
    print last_date
    
    last_index = overall_df.columns.tolist().index(last_date)
    
    test_feats = o_df_h.progress_apply(get_test_feats, axis = 1, last_index = last_index)
    
    test_feats["month"] = last_date.split("-")[1]
    test_feats["datenum"] = last_date.split("-")[2]
    test_feats["weekday"] = datetime.datetime.strptime(last_date, '%Y-%m-%d').date().weekday()

    prediction = get_preds(test_feats)
    
    o_df_h[last_date] = prediction
    #     preds = pd.concat([preds, prediction])
    








  0%|          | 0/5 [00:00<?, ?it/s]






6it [00:00, 820.00it/s]              

2017-01-01


/Library/Python/2.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy







  0%|          | 0/5 [00:00<?, ?it/s]






6it [00:00, 988.80it/s]              






  0%|          | 0/5 [00:00<?, ?it/s]






6it [00:00, 811.57it/s]              

2017-01-02
2017-01-03









  0%|          | 0/5 [00:00<?, ?it/s]






6it [00:00, 858.96it/s]              

...








  0%|          | 0/5 [00:00<?, ?it/s]






6it [00:00, 945.62it/s]              






  0%|          | 0/5 [00:00<?, ?it/s]






6it [00:00, 1007.72it/s]             

2017-01-13
2017-01-14









  0%|          | 0/5 [00:00<?, ?it/s]






6it [00:00, 799.58it/s]              






  0%|          | 0/5 [00:00<?, ?it/s]






6it [00:00, 893.26it/s]              

2017-01-15
2017-01-16









  0%|          | 0/5 [00:00<?, ?it/s]






6it [00:00, 750.84it/s]              

In [147]:
o_df_h

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2017-02-20,2017-02-21,2017-02-22,2017-02-23,2017-02-24,2017-02-25,2017-02-26,2017-02-27,2017-02-28,2017-03-01
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,1,1,1,1,1,1,1,1,1,1
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,1,1,1,1,1,1,1,1,1,1
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,1,1,1,1,1,1,1,1,1,1
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,1,1,1,1,1,1,1,1,1,1
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1


In [19]:
train_melt = pd.read_csv("train_feats.csv")

In [ ]:
len(test_feats)

In [62]:
train_melt.to_csv("train_feats.csv", index = False)

In [86]:
key["trunc_page"] = key.apply(lambda x: x["Page"][:-11], axis = 1)
key.to_csv("key_trunc.csv", index = False)

In [ ]:
# train_melt = pd.read_csv("train_feats.csv")
key = pd.read_csv("key_trunc.csv")

train_melt = pd.merge(train_melt, key, left_on="Page", right_on="trunc_page", how="inner")
train_melt.drop("Page_x", inplace=True, axis = 1)
train_melt.drop("Page_y", inplace=True, axis = 1)

train_melt.to_csv("train_merged.csv", index = False)

In [93]:
train_merged_filtered = train_merged[["Id", "median_60_days"]]

In [111]:
sub = pd.merge(sub, train_merged_filtered, on = "Id")
sub.drop("Visits", inplace=True, axis = 1)
sub.columns = ["Id", "Visits"]

In [112]:
sub.to_csv("median_60_days.csv", index = False)

In [ ]:
def run_xgb(x_train, x_test, x_label):

    # x_train = pd.concat([pos_train, neg_train]) #Concat positive and negative
    # y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist() #Putting in 1 and 0

    # x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    params['eta'] = 0.05
    params['max_depth'] = 6
    params['silent'] = 1

    d_train = xgb.DMatrix(x_train, label=x_label)
    d_test = xgb.DMatrix(x_test)

    watchlist = [(d_train, 'train')]

    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=50, verbose_eval=50)

    p_test = bst.predict(d_test)

    # xgb.plot_importance(bst)
    # pyplot.show()

    return p_test